In [7]:

# Import necessary libraries
import numpy as np
from itertools import izip  # For Python 2 compatibility, use `zip` if on Python 3
import os

# Configurations (modify as needed)
class Config:
    validation_fraction = 0.2  # Fraction of data used for validation
    batch_size = 32  # Batch size

config = Config()

# Define data path
data_path = 'data/data_v2'
assert os.path.exists(data_path), "Data path does not exist!"


In [ ]:

# Load data from the specified path
with open(data_path, 'r') as f:
    data = f.read()

# Split the data into tokens and initialize the token set
tokens_set = set(data.split())
start_symbol, end_symbol = '<s>', '</s>'
tokens_set.update({start_symbol, end_symbol})

# Create vocabulary mappings
idx2token = list(tokens_set)
vocab_size = len(idx2token)
print('Vocabulary size:', vocab_size)
token2idx = dict(izip(idx2token, range(vocab_size)))  # Use zip for Python 3


In [ ]:

# Process the tunes and tokenize them
tunes = data.split('\n\n')
del data  # Free up memory

tunes = [
    [token2idx[c] for c in [start_symbol] + t.split() + [end_symbol]]
    for t in tunes
]

# Sort tunes by length (longest first)
tunes.sort(key=lambda x: len(x), reverse=True)
ntunes = len(tunes)
print('Number of tunes:', ntunes)


In [ ]:
display(tunes[0])

In [ ]:

# Calculate tune lengths
tune_lens = np.array([len(t) for t in tunes])
max_len = max(tune_lens)
print('Max tune length:', max_len)

# Calculate the number of validation tunes
nvalid_tunes = ntunes * config.validation_fraction
nvalid_tunes = config.batch_size * max(
    1, int(round(nvalid_tunes / float(config.batch_size)))
)  # Round to a multiple of batch_size
print('Number of validation tunes:', nvalid_tunes)

# Split into training and validation sets
rng = np.random.RandomState(42)  # Fixed seed for reproducibility
valid_idxs = rng.choice(np.arange(ntunes), int(nvalid_tunes), replace=False)

# Create validation and training datasets
valid_tunes = [tunes[i] for i in valid_idxs]
train_tunes = [tunes[i] for i in range(ntunes) if i not in valid_idxs]

print('Training tunes:', len(train_tunes))
print('Validation tunes:', len(valid_tunes))


In [ ]:
data_path = 'DutchFolkTunes/dataset.txt'
assert os.path.exists(data_path), "Data path does not exist!"


# Load data from the specified path
with open(data_path, 'r') as f:
    data = f.read()

# Split the data into tokens and initialize the token set
tokens_set = set(data.split())
start_symbol, end_symbol = '<s>', '</s>'
tokens_set.update({start_symbol, end_symbol})

# Create vocabulary mappings
idx2token = list(tokens_set)
vocab_size = len(idx2token)
print('Vocabulary size:', vocab_size)
token2idx = dict(izip(idx2token, range(vocab_size)))  # Use zip for Python 3

# Process the tunes and tokenize them
tunes = data.split('\n\n')
del data  # Free up memory

tunes = [
    [token2idx[c] for c in [start_symbol] + t.split() + [end_symbol]]
    for t in tunes
]

# Sort tunes by length (longest first)
tunes.sort(key=lambda x: len(x), reverse=True)
ntunes = len(tunes)
print('Number of tunes:', ntunes)


In [ ]:
# Calculate tune lengths
tune_lens = np.array([len(t) for t in tunes])
max_len = max(tune_lens)
print('Max tune length:', max_len)

# Calculate the number of validation tunes
nvalid_tunes = ntunes * config.validation_fraction
nvalid_tunes = config.batch_size * max(
    1, int(round(nvalid_tunes / float(config.batch_size)))
)  # Round to a multiple of batch_size
print('Number of validation tunes:', nvalid_tunes)

# Split into training and validation sets
rng = np.random.RandomState(42)  # Fixed seed for reproducibility
valid_idxs = rng.choice(np.arange(ntunes), int(nvalid_tunes), replace=False)

# Create validation and training datasets
valid_tunes = [tunes[i] for i in valid_idxs]
train_tunes = [tunes[i] for i in range(ntunes) if i not in valid_idxs]

print('Training tunes:', len(train_tunes))
print('Validation tunes:', len(valid_tunes))


In [ ]:

# Visualization imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

# Set up plot styles
sns.set(style="whitegrid")

# Prepare a pandas DataFrame for tune lengths
tune_stats = pd.DataFrame({
    'Tune Length': tune_lens,
    'Tune Index': range(len(tune_lens))
})

# Display basic statistics in a DataFrame
display(tune_stats.describe())

# Plot a histogram of tune lengths
plt.figure(figsize=(10, 6))
sns.distplot(tune_stats['Tune Length'], kde=True, bins=30, color='skyblue')
plt.title("Distribution of Tune Lengths", fontsize=16)
plt.xlabel("Tune Length", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.show()

# Plot a box plot for tune lengths
plt.figure(figsize=(8, 6))
sns.boxplot(x=tune_stats['Tune Length'], color='lightgreen')
plt.title("Spread of Tune Lengths", fontsize=16)
plt.xlabel("Tune Length", fontsize=12)
plt.show()

# Token frequency analysis
token_freq = pd.Series(data=0, index=idx2token)
for tune in tunes:
    for idx in tune:
        token_freq[idx2token[idx]] += 1

# Display top 10 tokens in a DataFrame
top_tokens = token_freq.sort_values(ascending=False).head(30)
display(pd.DataFrame(top_tokens, columns=['Frequency']))

# Bar chart of top 10 tokens
plt.figure(figsize=(10, 6))
top_tokens.plot(kind='bar', color='orange')
plt.title("Top 10 Tokens by Frequency", fontsize=16)
plt.xlabel("Token", fontsize=12)
plt.ylabel("Frequency", fontsize=12)
plt.xticks(rotation=45)
plt.show()


In [ ]:
display(tune_stats.head(10))

### Shows what the output looks like


In [ ]:
# Import necessary libraries
import pickle
import pandas as pd

# Specify the path to your .pkl file
pkl_file_path = 'metadata/folkrnn_v2.pkl'

# Load the pickle file
with open(pkl_file_path, 'rb') as file:
    data = pickle.load(file)

display(data)


In [19]:
import re

# Path to the input text file
input_file = "DutchFolkTunes\dataset.txt"  # Replace with your file path

# Read the content of the input file
with open(input_file, "r") as file:
    input_string = file.read()


# Regular expression to find patterns like [M ... ], [K ... ], or [L ... ]
pattern = r"(\[M.*?\])|(\[K.*?\])|(\[L.*?\])"

# Add a newline before the matched patterns
modified_string = re.sub(pattern, lambda m: 
    '\n' + (m.group(2) + '\n' if m.group(2) else m.group(1) if m.group(1) else m.group(3)), 
    input_string)

modified_string = re.sub(pattern, lambda m: (
    '\n' + (m.group(2) + '\n' if m.group(2) else  # [K ... ]: add newline after
    ('\n' + m.group(3) if m.group(3) else  # [L ... ]: add newline before
    m.group(1)))  # [M ... ]: no changes
), input_string)


# Path to the output text file
output_file = "DutchFolkTunes\output.txt"  # Replace with your desired output file path

# Write the modified content to the output file
with open(output_file, "w") as file:
    file.write(modified_string)



In [3]:
# Import required libraries
import re

# Function to process lines
def process_lines(input_file, output_file):
    with open(input_file, 'r') as file:
        lines = file.readlines()
    
    processed_lines = []
    
    for line in lines:
        # Use a regular expression to find two consecutive numbers (e.g., "23") and remove everything after
        match = re.search(r'\d{2,}', line)  # Matches any sequence of 2 or more digits
        if match:
            line = line[:match.start()]  # Keep only content before the match
        processed_lines.append(line.strip())
    
    # Save the processed lines to a new file
    with open(output_file, 'w') as file:
        for processed_line in processed_lines:
            file.write(processed_line + '\n')
    
    #print(f"Processed lines saved to {output_file}")

# Example usage
input_filename = "DutchFolkTunes\output.txt"   # Replace with your input file name
output_filename = "DutchFolkTunes\Finaloutput.txt" # Replace with your desired output file name
process_lines(input_filename, output_filename)


In [ ]:
# Define the input and output file paths
input_file = "DutchFolkTunes\output.txt"  # Replace with your input text file
output_file = "DutchFolkTunes\testoutput.txt"  # Replace with your desired output text file

# Open the input file and process the lines
with open(input_file, 'r') as file:
    lines = file.readlines()

# Create a list to store the filtered lines
filtered_lines = []

# Loop through the lines and apply the condition
for line in lines:
    # Check if the line starts with [M and does not end with ]
    if not (line.startswith("[M") and not line.rstrip().endswith("]")):
        filtered_lines.append(line)

# Write the filtered lines to the output file
with open(output_file, 'w') as file:
    file.writelines(filtered_lines)